# Generalized Advantage Estimate and Discounted Returns

The function call for Generalized Advantage Estimate (GAE)/discounted returns is usually in the form of :

```python
def compute_discount(rewards, gamma, lamda, masks, next_value):
    ...
```
The function inputs are:
* **rewards**: a list containing collected rewards.
* **gamma**: discount factor $\gamma \in [0,1.0]$. Usually set at 0.9, 0.95, 0.99.
* **lamda**: weighting factor $\lambda \in [0, 1.0]$ for GAE. Usually set at 0.95
* **masks**: a list containing masking for termination of episode. We set masks to be 0 if done, else set to 1.0.
* **next_value**: next value prediction provided so we can use it to bootstrap our computation of discount returns. See below for more details.

### Example: CartPole
Let's see how this works, Let's take an arbitrary example using OpenAI Gym's CartPole. Say we ran our agent in the CartPole environment for 10 steps, resulting in the following collected data:
```python
num_steps = 10
gamma = 1.0
lam = 1.0
rewards = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
masks = [1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
```
We can see from above that we collected data across two episodes (by looking at ```masks```). The first episode ran for 6 steps, terminating at ```masks[5]```. We then collected 4 more steps of data for the second episode (the environment resets internally when one episode terminates). **Note:** we don't know whether the last step collected above is the terminal step for the second episode or not. We will discuss this below.

Let's first get a sense of what our returns should be. (Remember: **returns** is the **discounted sum of future rewards** from each time step). Given that we don't do any discounting (i.e ```gamma``` is set to 1.0), our returns for the first episode should be:
```python
return_for_episode_1 = [6.0, 5.0, 4.0, 3.0, 2.0, 1.0]
```
(Remember: **return** is the **discounted sum of future rewards** from each time step)

What about for the next episode? Since we only collected 4 steps of the next episode, we can't see far enough to see whether the next episode only contains 4 steps or more. This is where **next_value** comes into play. **next_value** is used to bootstrap our return estimate.

Imagine two scenarios, in the first scenario, the second episode terminates right after the 4 steps, so we get one more reward of [1.0] and the episode terminates. Then we know our return for second episode is:
```python
return_for_episode_1 = [6.0, 5.0, 4.0, 3.0, 2.0, 1.0]
next_value = 1.0 # next step is terminal step, with last reward of 1.0
return_for_episode_2 = [5.0, 4.0, 3.0, 2.0]
returns_for_all_10_steps = [6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 5.0, 4.0, 3.0, 2.0]
```
In the second scenario, our 2nd episode goes for another 10 steps before termination.
```python
return_for_episode_1 = [6.0, 5.0, 4.0, 3.0, 2.0, 1.0]
next_value = 10.0
return_for_episode_2 = [14.0, 13.0, 12.0, 11.0]
returns_for_all_10_steps = [6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 14.0, 13.0, 12.0, 11.0]
```
**Note**: In both scenarios above, we assumed to know exactly how many further steps and rewards recieved before termination (by providing the exact **next_value**). In reality, we won't know during our data collection until we have completed the entire 2nd episode rollout. To be able to get an immediate estimate of the next_value, we often parameterize a value function ```V(s)``` that estimates the next_value, this will introduce some bias/error in our return estimate. But this is the price we pay for bootstrapping (i.e get an estimate of next_value without waiting to finish collecting data for the next episode).

## Discounted Return Computation (First Without GAE)

Below is code for computing discounted return from the above cartpole example. Note that we utilize our masks list to take into account episode termination. This will reset the return count (by setting return accumulated so far to 0, making the termination step's return effectly = reward for that step):

In [22]:
import numpy as np

def compute_discounted_return(rewards, masks, gamma, next_value):
    """Compute discounted returns given a list of rewards, discount_factor, a list of termination masks, 
    and next value estiamte.
    
    """
    data_len = len(rewards) # get length of episode
    returns = np.zeros(data_len+1) # create returns, with additional index for next_value
    returns[-1] = next_value # set additional index as next_value
    
    for i in reversed(range(data_len)):
        returns[i] = rewards[i] + gamma*returns[i+1]*masks[i]
    # note that returns should exclude the additional index.
    return returns[:-1]
    
    
# CartPole Example
num_steps = 10
gamma = 1.0
rewards = np.ones(num_steps) # reward of all 1.0s
masks = np.ones(num_steps) # create our masks
masks[5] = 0.0 # set step 6 to be 0.0, indicating end of episode 1.
next_value_case_1 = 1.0 # next step is termination for episode 2.
next_value_case_2 = 10.0 # next step is not termination for episode 2.

returns_case_1 = compute_discounted_return(rewards, masks, gamma, next_value_case_1)
returns_case_2 = compute_discounted_return(rewards, masks, gamma, next_value_case_2)
print('return_case_1 is {} \nreturn_case_2 is {}'.format(returns_case_1, returns_case_2))

return_case_1 is [ 6.  5.  4.  3.  2.  1.  5.  4.  3.  2.] 
return_case_2 is [  6.   5.   4.   3.   2.   1.  14.  13.  12.  11.]


---
## Generalized Advantage Estimate ([GAE](https://arxiv.org/pdf/1506.02438.pdf))

See reference for GAE [here](https://arxiv.org/pdf/1506.02438.pdf).

$GAE(\gamma,\lambda)$ is exponential-weight average of k-step Advantage estimators (Huh?). It's important to note here what exactly is Advantage $A^{\pi,\gamma}(s_t,a_t)$. **FYI:** $A^{\pi,\gamma}(s_t,a_t)$ reads as the Advantage ($A$) estimate for being in state ($s_t$), taking action ($a_t$), while following policy ($\pi$) and using discount factor ($\gamma$).

Formally: $A^{\pi,\gamma}(s_t,a_t) = Q^{\pi,\gamma}(s_t,a_t) - V^{\pi,\gamma}(s_t,a_t)$

Where $Q^{\pi,\gamma}(s_t,a_t)$ is the state-action value function (Also called Q-function) and $ V^{\pi,\gamma}(s_t,a_t)$ is the state value function (Also called V-function).

Intuitively (subjective): Q-function estimates the value of being in some state and taking some action, where as V-function estimates the value of being in some state, by taking A = Q - V, we say Advantage estimate the advantage of the action taking given being in some state.

### Temporal Difference Residual for V

Previously, by the definition of A, it would seem that we need to have both Q and V functions to estimate A. We can simply this a bit by considering the fact that given a sufficient accurate V-function, we get a biased estimate for our advantage using only V-funcion alone(via the TD residual ):

$\mathbb{E}_{s_{t+1}}\left[ \hat{A}_{t} \right] = \mathbb{E}_{s_{t+1}} $


